# Compliance Checks
To what degree do the models do or refuse their tasks?

In [ ]:
# What is the experiment with the base completions we want to use?
STUDY_FOLDER = "english_words_basic" # 🔵 within exp/
CONDITIONS = { 
    # see `analysis/loading_data.py` for details
    # "language_model": ["gpt-3.5-turbo", "gpt-4-turbo"],
}

In [ ]:
from pathlib import Path
import subprocess
import sys

In [ ]:
# Run the git command to get the repository root directory
REPO_DIR = subprocess.check_output(["git", "rev-parse", "--show-toplevel"]).decode().strip()

print("Repository directory:", REPO_DIR)
sys.path.append(REPO_DIR)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import words
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from analysis_helpers import merge_base_and_self_pred_dfs, create_df_from_configs, fill_df_with_function
from loading_data import load_dfs_with_filter

In [ ]:
# Set the display option to None to show all content
pd.set_option('display.max_colwidth', None)

In [ ]:
# set color palette
palette = sns.color_palette("Set1")
sns.set_palette(palette)

In [ ]:
# Set the directory for the data
EXPDIR = Path(REPO_DIR) / "exp"

In [ ]:
# load the dataframes with configs as keys while keeping the non-compliant ones
dfs = load_dfs_with_filter(EXPDIR / STUDY_FOLDER, CONDITIONS, exclude_noncompliant=False)

In [ ]:
def is_base_config(config):
    return "base" in config["prompt"]["method"]

In [ ]:
base_dfs = {config: df for config, df in dfs.items() if is_base_config(config)}
self_pred_dfs = {config: df for config, df in dfs.items() if not is_base_config(config)}
print(f"Loaded {len(base_dfs)} base and {len(self_pred_dfs)} self-prediction dataframes")

In [ ]:
# create results dataframe
results = create_df_from_configs(dfs.keys())

How many responses are non-compliant?

In [ ]:
def avg_compliance(df):
    return (df["compliance"] == True).mean()

In [ ]:
# fill the results dataframe with the accuracy and t-statistic
fill_df_with_function(dfs, avg_compliance, "avg_compliance", results)

In [ ]:
results.drop(columns=["config"]).sort_values(by="avg_compliance", ascending=False).style.hide(axis="index").background_gradient(subset=["avg_compliance"], cmap="RdYlGn")

Making plots

In [ ]:
sns.pointplot(data=results, x="dataset_n_shot", y="avg_compliance", hue="language_model")
plt.title("Average compliance per number of few-shot examples")
plt.xlabel("Number of few-shot examples")
plt.ylabel("Average compliance")
plt.ylim(-0.025, 1.025)
plt.show()


In [ ]:
# a bar plot with bars for each language model and prompt method and also n_shot
sns.barplot(data=results, x="language_model", y="avg_compliance", hue="prompt_method")
plt.title("Average compliance per language model and prompt method")
plt.xlabel("Language model")
plt.ylabel("Average compliance")
plt.show()
